In [1]:
import gzip
import json
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [3]:
objs0=[]
with gzip.open("dataset0.json.gz", 'rt') as f:
    for line in f:
        objs0.append(json.loads(line))



rows0 = []
for i in range(len(objs0)):
    for key1, inner_dict in objs0[i].items():
        for key2, inner_list in inner_dict.items():
            for key3, sublist in inner_list.items():
                for subsublist in sublist:
                    rows0.append([key1, key2, key3] + subsublist)
                    
df0 = pd.DataFrame(rows0, columns=["transcript_id", "transcript_position", "combined nucleotides", "dwelling_time1", "sd1", "mean1", "dwelling_time2", "sd2", "mean2", "dwelling_time3", "sd3", "mean3"])
df0['transcript_position'] = df0['transcript_position'].astype(int)


#df0info = pd.merge(df0, data, on=['transcript_id', 'transcript_position'], how='inner')
 

KeyboardInterrupt: 

In [ ]:
objs1=[]
with gzip.open("dataset1.json.gz", 'rt') as f:
    for line in f:
        objs1.append(json.loads(line))
        
rows1 = []
for i in range(len(objs1)):
    for key1, inner_dict in objs1[i].items():
        for key2, inner_list in inner_dict.items():
            for key3, sublist in inner_list.items():
                for subsublist in sublist:
                    rows1.append([key1, key2, key3] + subsublist)                    
df1 = pd.DataFrame(rows1, columns=["transcript_id", "transcript_position", "combined nucleotides", "dwelling_time1", "sd1", "mean1", "dwelling_time2", "sd2", "mean2", "dwelling_time3", "sd3", "mean3"])
df1['transcript_position'] = df1['transcript_position'].astype(int)
#df1info = pd.merge(df1, data, on=['transcript_id', 'transcript_position'], how='inner')


In [ ]:
objs2=[]
with gzip.open("dataset2.json.gz", 'rt') as f:
    for line in f:
        objs2.append(json.loads(line))

rows2 = []
for i in range(len(objs2)):
    for key1, inner_dict in objs2[i].items():
        for key2, inner_list in inner_dict.items():
            for key3, sublist in inner_list.items():
                for subsublist in sublist:
                    rows2.append([key1, key2, key3] + subsublist)     

df2 = pd.DataFrame(rows2, columns=["transcript_id", "transcript_position", "combined nucleotides", "dwelling_time1", "sd1", "mean1", "dwelling_time2", "sd2", "mean2", "dwelling_time3", "sd3", "mean3"])
df2['transcript_position'] = df2['transcript_position'].astype(int)

In [5]:
df0.to_pickle('df0.pkl')
df1.to_pickle('df1.pkl')
df2.to_pickle('df2.pkl')

In [2]:
df0 = pd.read_pickle("df0.pkl")
df1 = pd.read_pickle("df1.pkl")
df2 = pd.read_pickle("df2.pkl")


In [3]:
og_df = df1
og_df


,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3
0,ENST00000000233,244,AAGACCA,0.00465,2.16,127.0,0.00640,3.90,127.0,0.00797,8.75,83.7
1,ENST00000000233,244,AAGACCA,0.02690,4.43,106.0,0.01860,10.00,123.0,0.00863,6.20,80.0
2,ENST00000000233,244,AAGACCA,0.00432,3.10,108.0,0.01200,8.26,125.0,0.01590,2.89,78.7
3,ENST00000000233,244,AAGACCA,0.00996,4.52,123.0,0.01750,8.51,128.0,0.00498,2.63,80.0
4,ENST00000000233,244,AAGACCA,0.00764,2.81,124.0,0.00772,4.22,126.0,0.00474,5.84,80.9
...,...,...,...,...,...,...,...,...,...,...,...,...
7907947,ENST00000641784,3266,CTAACTT,0.00589,1.20,93.5,0.00270,1.58,102.0,0.01530,2.19,92.3
7907948,ENST00000641784,3266,CTAACTT,0.00401,2.00,92.1,0.00764,3.00,99.1,0.00469,1.37,92.6
7907949,ENST00000641784,3266,CTAACTT,0.00430,1.32,91.8,0.00270,1.33,100.0,0.00550,1.37,93.9
7907950,ENST00000641784,3266,CTAACTT,0.00280,2.70,90.7,0.00564,1.45,100.0,0.01600,1.90,92.3


In [4]:
count_df1 = og_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='read_count')
count_df2 = og_df.groupby(['transcript_id']).size().reset_index(name='expression_count') #most genes are likely to produce unique transcripts
df = og_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
df = df.merge(count_df2, on=['transcript_id'])

agg_functions = {
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
}

summary_df = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()

summary_df = summary_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
summary_df = summary_df.merge(count_df2, on=['transcript_id'])

#summary_df['label_percentage'] = summary_df['label'] / summary_df['read_count'] * 100
#summary_df = summary_df.rename(columns={'label': 'label_sum'})
#summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

In [5]:
# function to convert features
def sequence_to_features(seq):
    """
    Convert a 7-letter sequence into binary features for the first and last letters.
    
    Parameters:
    - seq: A string of 7 letters (A, T, C, G)
    
    Returns:
    - A dictionary with binary features for the first and last letters.
    """
    letters = ['A', 'T', 'C', 'G']
    features = {}
    
    first_char = seq[0]
    last_char = seq[-1]
    
    for letter in letters:
        features[f"1{letter}"] = first_char == letter
        features[f"7{letter}"] = last_char == letter
            
    return features

In [6]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [7]:
new_df['5-mer'] = new_df['combined nucleotides'].apply(lambda x: x[1:-1])
new_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,ENST00000000233,244,AAGACCA,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,...,3159,True,True,False,False,False,False,False,False,AGACC
1,ENST00000000233,261,CAAACTG,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,...,3159,False,False,False,False,True,False,False,True,AAACT
2,ENST00000000233,316,GAAACAG,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,...,3159,False,False,False,False,False,False,True,True,AAACA
3,ENST00000000233,332,AGAACAT,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,...,3159,True,False,False,True,False,False,False,False,GAACA
4,ENST00000000233,368,AGGACAA,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,...,3159,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90805,ENST00000641834,1348,GGGACAT,0.010009,3.119023,118.167630,0.008217,4.816532,117.213873,0.007304,...,2880,False,False,False,True,False,False,True,False,GGACA
90806,ENST00000641834,1429,CTGACAC,0.008481,4.634631,110.052349,0.012667,8.901409,114.590604,0.005008,...,2880,False,False,False,False,True,True,False,False,TGACA
90807,ENST00000641834,1531,TGGACAC,0.007289,3.702195,113.851562,0.007072,4.365023,114.125000,0.006613,...,2880,False,False,True,False,False,True,False,False,GGACA
90808,ENST00000641834,1537,CTGACCA,0.006971,3.148815,109.843704,0.006916,6.331704,122.340741,0.006476,...,2880,False,True,False,False,True,False,False,False,TGACC


In [8]:
combined = new_df.copy()
label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,0,244,AAGACCA,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,...,3159,True,True,False,False,False,False,False,False,AGACC
1,0,261,CAAACTG,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,...,3159,False,False,False,False,True,False,False,True,AAACT
2,0,316,GAAACAG,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,...,3159,False,False,False,False,False,False,True,True,AAACA
3,0,332,AGAACAT,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,...,3159,True,False,False,True,False,False,False,False,GAACA
4,0,368,AGGACAA,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,...,3159,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90805,4450,1348,GGGACAT,0.010009,3.119023,118.167630,0.008217,4.816532,117.213873,0.007304,...,2880,False,False,False,True,False,False,True,False,GGACA
90806,4450,1429,CTGACAC,0.008481,4.634631,110.052349,0.012667,8.901409,114.590604,0.005008,...,2880,False,False,False,False,True,True,False,False,TGACA
90807,4450,1531,TGGACAC,0.007289,3.702195,113.851562,0.007072,4.365023,114.125000,0.006613,...,2880,False,False,True,False,False,True,False,False,GGACA
90808,4450,1537,CTGACCA,0.006971,3.148815,109.843704,0.006916,6.331704,122.340741,0.006476,...,2880,False,True,False,False,True,False,False,False,TGACC


In [9]:
combined = pd.get_dummies(combined, columns=['5-mer'], drop_first=False)
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,AAGACCA,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,...,False,False,False,False,False,False,False,False,False,False
1,0,261,CAAACTG,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,...,False,False,False,False,False,False,False,False,False,False
2,0,316,GAAACAG,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,...,False,False,False,False,False,False,False,False,False,False
3,0,332,AGAACAT,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,...,False,False,False,False,False,False,False,False,False,False
4,0,368,AGGACAA,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90805,4450,1348,GGGACAT,0.010009,3.119023,118.167630,0.008217,4.816532,117.213873,0.007304,...,False,True,False,False,False,False,False,False,False,False
90806,4450,1429,CTGACAC,0.008481,4.634631,110.052349,0.012667,8.901409,114.590604,0.005008,...,False,False,False,False,False,False,False,True,False,False
90807,4450,1531,TGGACAC,0.007289,3.702195,113.851562,0.007072,4.365023,114.125000,0.006613,...,False,True,False,False,False,False,False,False,False,False
90808,4450,1537,CTGACCA,0.006971,3.148815,109.843704,0.006916,6.331704,122.340741,0.006476,...,False,False,False,False,False,False,False,False,True,False


In [10]:
combined = combined.drop(columns=['combined nucleotides'])
combined

,transcript_id,transcript_position,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,0.008722,3.990545,123.090909,0.010218,6.716545,125.503030,0.006773,3.999745,...,False,False,False,False,False,False,False,False,False,False
1,0,261,0.006292,2.849892,109.027711,0.007084,3.197289,107.190964,0.007168,2.686892,...,False,False,False,False,False,False,False,False,False,False
2,0,316,0.007237,2.782710,105.440237,0.007050,3.463964,98.539053,0.007141,2.083775,...,False,False,False,False,False,False,False,False,False,False
3,0,332,0.010299,6.042386,129.304348,0.008195,2.733641,97.320652,0.005806,2.252565,...,False,False,False,False,False,False,False,False,False,False
4,0,368,0.011535,6.031751,118.000000,0.012108,5.748927,121.485876,0.009484,3.958192,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90805,4450,1348,0.010009,3.119023,118.167630,0.008217,4.816532,117.213873,0.007304,3.959769,...,False,True,False,False,False,False,False,False,False,False
90806,4450,1429,0.008481,4.634631,110.052349,0.012667,8.901409,114.590604,0.005008,3.505034,...,False,False,False,False,False,False,False,True,False,False
90807,4450,1531,0.007289,3.702195,113.851562,0.007072,4.365023,114.125000,0.006613,2.339922,...,False,True,False,False,False,False,False,False,False,False
90808,4450,1537,0.006971,3.148815,109.843704,0.006916,6.331704,122.340741,0.006476,2.578881,...,False,False,False,False,False,False,False,False,True,False


In [11]:
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek

# ADASYN followed by Tomek Links Pipeline
adasyn_tomek = Pipeline([
    ('adasyn', ADASYN(random_state=42)),
    ('tomek', TomekLinks())
])

# Dictionary of resampling methods
resampling_methods = {
    'RandomUnderSampler': RandomUnderSampler(random_state=42),
    'TomekLinks': TomekLinks(),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42),
    'ADASYN_Tomek': adasyn_tomek
}

In [12]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

X1_min_features = combined.drop(columns=['transcript_id', 'transcript_position'])
X2_min_features = combined.drop(columns=['transcript_id'])
    

In [25]:
model1 = pickle.load(open('xgb_model_1.pkl','rb'))
model2 = pickle.load(open('xgb_model_2.pkl','rb'))
X1_predicted = model1.predict_proba(X1_min_features)[:, 1]
X2_predicted = model2.predict_proba(X2_min_features)[:, 1]


In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_min_features_non_bool = X1_min_features.select_dtypes(exclude=['bool'])
X1_min_features_bool = X1_min_features.select_dtypes(include=['bool'])
x1_non_bool_scaled = scaler.fit_transform(X1_min_features_non_bool)
x1_final = np.hstack([x1_non_bool_scaled,X1_min_features_bool.values])
x1_final_proba = model1.predict_proba(x1_final)[:,1]

X2_min_features_non_bool = X2_min_features.select_dtypes(exclude=['bool'])
X2_min_features_bool = X2_min_features.select_dtypes(include=['bool'])
x2_non_bool_scaled = scaler.fit_transform(X2_min_features_non_bool)
x2_final = np.hstack([x2_non_bool_scaled,X2_min_features_bool.values])
x2_final_proba = model2.predict_proba(x2_final)[:,1]

summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = x1_final_proba.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('team1_dataset1_1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= x2_final_proba.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('team1_dataset1_2.csv',index=False)

In [31]:
#X1_scaled = scaler.fit_transform(X1_min_features)
#X1_scaledpred = model1.predict_proba(X1_scaled)[:, 1]
#X2_scaled = scaler.fit_transform(X2_min_features)
#X2_scaledpred = model2.predict_proba(X2_scaled)[:, 1]
summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = x1_final_proba.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('team1_dataset1_1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= x2_final_proba.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('team1_dataset1_2.csv',index=False)

In [20]:
summary_df_final1 = summary_df.copy()
summary_df_final1['score'] =X1_predicted.tolist()

In [21]:
summary_df_final2 = summary_df.copy()
summary_df_final2['score']= X2_predicted.tolist()

In [22]:
summary_df_final1[["transcript_id","transcript_position","score"]].to_csv('dataset1csv1.csv',index=False)

In [23]:
summary_df_final2[["transcript_id","transcript_position","score"]].to_csv('dataset1csv2.csv',index=False)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------<br>

In [33]:
og_df = df0
og_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3
0,ENST00000000233,244,AAGACCA,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2
...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4
11027102,ENST00000641834,1693,TTGACAT,0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6
11027103,ENST00000641834,1693,TTGACAT,0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3
11027104,ENST00000641834,1693,TTGACAT,0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8


In [34]:
count_df1 = og_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='read_count')
count_df2 = og_df.groupby(['transcript_id']).size().reset_index(name='expression_count') #most genes are likely to produce unique transcripts
df = og_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
df = df.merge(count_df2, on=['transcript_id'])

agg_functions = {
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
}

summary_df = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()

summary_df = summary_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
summary_df = summary_df.merge(count_df2, on=['transcript_id'])

#summary_df['label_percentage'] = summary_df['label'] / summary_df['read_count'] * 100
#summary_df = summary_df.rename(columns={'label': 'label_sum'})
#summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

In [35]:
summary_df


,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,185,3449
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,172,3449
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,185,3449
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,200,3449
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,198,3449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,73,1323
121834,ENST00000641834,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,69,1323
121835,ENST00000641834,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,84.190625,64,1323
121836,ENST00000641834,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,57,1323


In [36]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [37]:
new_df['5-mer'] = new_df['combined nucleotides'].apply(lambda x: x[1:-1])
new_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,3449,True,True,False,False,False,False,False,False,AGACC
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,3449,False,False,False,False,True,False,False,True,AAACT
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,3449,False,False,False,False,False,False,True,True,AAACA
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,3449,True,False,False,True,False,False,False,False,GAACA
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,3449,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,1323,False,False,False,True,False,False,True,False,GGACA
121834,ENST00000641834,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,1323,False,False,False,False,True,True,False,False,TGACA
121835,ENST00000641834,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,1323,False,False,True,False,False,True,False,False,GGACA
121836,ENST00000641834,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,1323,False,True,False,False,True,False,False,False,TGACC


In [38]:
combined = new_df.copy()
label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,3449,True,True,False,False,False,False,False,False,AGACC
1,0,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,3449,False,False,False,False,True,False,False,True,AAACT
2,0,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,3449,False,False,False,False,False,False,True,True,AAACA
3,0,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,3449,True,False,False,True,False,False,False,False,GAACA
4,0,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,3449,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,1323,False,False,False,True,False,False,True,False,GGACA
121834,5332,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,1323,False,False,False,False,True,True,False,False,TGACA
121835,5332,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,1323,False,False,True,False,False,True,False,False,GGACA
121836,5332,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,1323,False,True,False,False,True,False,False,False,TGACC


In [39]:
combined = pd.get_dummies(combined, columns=['5-mer'], drop_first=False)
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,False,False,False,False,False,False,False
1,0,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,False,False,False,False,False,False,False,False
2,0,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,False,False,False,False,False,False,False,False,False
3,0,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,False,False,False,False,False,False,False,False,False
4,0,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,False,False,False,False,False,True,False


In [40]:
combined = combined.drop(columns=['combined nucleotides'])
combined

,transcript_id,transcript_position,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,...,False,False,False,False,False,False,False,False,False,False
1,0,261,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,...,False,False,False,False,False,False,False,False,False,False
2,0,316,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,...,False,False,False,False,False,False,False,False,False,False
3,0,332,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,...,False,False,False,False,False,False,False,False,False,False
4,0,368,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,...,False,False,False,False,False,False,False,False,True,False


In [41]:
X1_min_features = combined.drop(columns=['transcript_id', 'transcript_position'])
X2_min_features = combined.drop(columns=['transcript_id'])

In [52]:
model1 = pickle.load(open('xgb_model_1.pkl','rb'))
model2 = pickle.load(open('xgb_model_2.pkl','rb'))
X1_predicted = model1.predict_proba(X1_min_features)[:, 1]
X2_predicted = model2.predict_proba(X2_min_features)[:, 1]


In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_min_features_non_bool = X1_min_features.select_dtypes(exclude=['bool'])
X1_min_features_bool = X1_min_features.select_dtypes(include=['bool'])
x1_non_bool_scaled = scaler.fit_transform(X1_min_features_non_bool)
x1_final = np.hstack([x1_non_bool_scaled,X1_min_features_bool.values])
x1_final_proba = model1.predict_proba(x1_final)[:,1]

X2_min_features_non_bool = X2_min_features.select_dtypes(exclude=['bool'])
X2_min_features_bool = X2_min_features.select_dtypes(include=['bool'])
x2_non_bool_scaled = scaler.fit_transform(X2_min_features_non_bool)
x2_final = np.hstack([x2_non_bool_scaled,X2_min_features_bool.values])
x2_final_proba = model2.predict_proba(x2_final)[:,1]

summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = x1_final_proba.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('team1_dataset0_1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= x2_final_proba.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('team1_dataset0_2.csv',index=False)

In [64]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y,X2_scaledpred)

0.9363198426024497

In [66]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_scaled = scaler.fit_transform(X1_min_features)
X1_scaledpred = model1.predict_proba(X1_scaled)[:, 1]
X2_scaled = scaler.fit_transform(X2_min_features)
X2_scaledpred = model2.predict_proba(X2_scaled)[:, 1]
summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = X1_scaledpred.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('dataset0scaled1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= X2_scaledpred.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('dataset0scaled2.csv',index=False)

In [35]:
summary_df_final1 = summary_df.copy()
summary_df_final1['score'] =X1_predicted.tolist()

In [74]:
summary_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count
0,ENST00000000233,244,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,185,3449
1,ENST00000000233,261,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,172,3449
2,ENST00000000233,316,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,185,3449
3,ENST00000000233,332,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,200,3449
4,ENST00000000233,368,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,198,3449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,73,1323
121834,ENST00000641834,1429,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,69,1323
121835,ENST00000641834,1531,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,84.190625,64,1323
121836,ENST00000641834,1537,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,57,1323


In [36]:
summary_df_final2 = summary_df.copy()
summary_df_final2['score']= X2_predicted.tolist()

In [37]:
summary_df_final1[["transcript_id","transcript_position","score"]].to_csv('dataset0csv1.csv',index=False)

In [38]:
summary_df_final2[["transcript_id","transcript_position","score"]].to_csv('dataset0csv2.csv',index=False)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------<br>

In [43]:
og_df = df2
og_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3
0,tx_id_0,0,AAAACCT,0.01220,3.99,106.0,0.00337,4.56,102.0,0.00664,4.200,84.2
1,tx_id_0,0,AAAACCT,0.03020,2.32,107.0,0.00443,2.36,102.0,0.00332,2.130,79.2
2,tx_id_0,0,AAAACCT,0.00232,5.55,110.0,0.00664,7.04,99.3,0.00232,2.210,86.6
3,tx_id_0,0,AAAACCT,0.00465,2.10,104.0,0.00996,3.90,108.0,0.00401,2.180,82.2
4,tx_id_0,0,AAAACCT,0.02110,3.49,103.0,0.00531,3.80,101.0,0.00997,2.180,81.2
...,...,...,...,...,...,...,...,...,...,...,...,...
1171935,tx_id_6,1880,CGAACAA,0.00675,8.33,111.0,0.00759,6.15,104.0,0.00896,2.420,91.6
1171936,tx_id_6,1880,CGAACAA,0.00529,4.81,106.0,0.00332,7.42,101.0,0.00996,1.940,90.9
1171937,tx_id_6,1880,CGAACAA,0.01460,7.72,104.0,0.00266,1.21,92.4,0.02150,2.110,90.0
1171938,tx_id_6,1880,CGAACAA,0.00610,4.21,119.0,0.00232,5.44,94.8,0.00199,2.000,82.6


In [44]:
count_df1 = og_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='read_count')
count_df2 = og_df.groupby(['transcript_id']).size().reset_index(name='expression_count') #most genes are likely to produce unique transcripts
df = og_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
df = df.merge(count_df2, on=['transcript_id'])

agg_functions = {
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
}

summary_df = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()

summary_df = summary_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
summary_df = summary_df.merge(count_df2, on=['transcript_id'])

#summary_df['label_percentage'] = summary_df['label'] / summary_df['read_count'] * 100
#summary_df = summary_df.rename(columns={'label': 'label_sum'})
#summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

In [45]:
summary_df


,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count
0,tx_id_0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,2.355356,81.607232,885,208026
1,tx_id_0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,3.988236,74.023667,1069,208026
2,tx_id_0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,2.625385,88.990429,1933,208026
3,tx_id_0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,3.803127,71.245575,678,208026
4,tx_id_0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,3.311097,91.278961,1868,208026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,tx_id_6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,2.780602,84.474332,748,233133
1319,tx_id_6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,3.420755,86.052461,711,233133
1320,tx_id_6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,4.575894,79.646198,526,233133
1321,tx_id_6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,2.832561,78.582196,601,233133


In [46]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [47]:
new_df['5-mer'] = new_df['combined nucleotides'].apply(lambda x: x[1:-1])
new_df

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,tx_id_0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,...,208026,True,False,False,True,False,False,False,False,AAACC
1,tx_id_0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,...,208026,False,False,True,False,False,True,False,False,GGACC
2,tx_id_0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,...,208026,False,True,False,False,False,False,True,False,GGACT
3,tx_id_0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,...,208026,False,True,True,False,False,False,False,False,GGACC
4,tx_id_0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,...,208026,False,True,True,False,False,False,False,False,AGACT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,tx_id_6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,...,233133,False,False,False,True,True,False,False,False,GAACC
1319,tx_id_6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,...,233133,False,False,False,False,True,False,False,True,AAACA
1320,tx_id_6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,...,233133,False,False,False,False,True,False,False,True,GGACA
1321,tx_id_6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,...,233133,False,True,False,False,True,False,False,False,AGACA


In [48]:
combined = new_df.copy()
label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,expression_count,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,...,208026,True,False,False,True,False,False,False,False,AAACC
1,0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,...,208026,False,False,True,False,False,True,False,False,GGACC
2,0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,...,208026,False,True,False,False,False,False,True,False,GGACT
3,0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,...,208026,False,True,True,False,False,False,False,False,GGACC
4,0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,...,208026,False,True,True,False,False,False,False,False,AGACT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,...,233133,False,False,False,True,True,False,False,False,GAACC
1319,6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,...,233133,False,False,False,False,True,False,False,True,AAACA
1320,6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,...,233133,False,False,False,False,True,False,False,True,GGACA
1321,6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,...,233133,False,True,False,False,True,False,False,False,AGACA


In [49]:
combined = pd.get_dummies(combined, columns=['5-mer'], drop_first=False)
combined

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,...,False,False,False,False,False,False,False,False,False,False
1,0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,...,False,False,True,False,False,False,False,False,False,False
2,0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,...,False,False,False,True,False,False,False,False,False,False
3,0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,...,False,False,True,False,False,False,False,False,False,False
4,0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,...,False,False,False,False,False,False,False,False,False,False
1319,6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,...,False,False,False,False,False,False,False,False,False,False
1320,6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,...,False,True,False,False,False,False,False,False,False,False
1321,6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,...,False,False,False,False,False,False,False,False,False,False


In [50]:
combined = combined.drop(columns=['combined nucleotides'])
combined

,transcript_id,transcript_position,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,0,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,2.355356,...,False,False,False,False,False,False,False,False,False,False
1,0,10,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,3.988236,...,False,False,True,False,False,False,False,False,False,False
2,0,20,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,2.625385,...,False,False,False,True,False,False,False,False,False,False
3,0,30,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,3.803127,...,False,False,True,False,False,False,False,False,False,False
4,0,40,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,3.311097,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,6,1840,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,2.780602,...,False,False,False,False,False,False,False,False,False,False
1319,6,1850,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,3.420755,...,False,False,False,False,False,False,False,False,False,False
1320,6,1860,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,4.575894,...,False,True,False,False,False,False,False,False,False,False
1321,6,1870,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,2.832561,...,False,False,False,False,False,False,False,False,False,False


In [51]:
X1_min_features = combined.drop(columns=['transcript_id', 'transcript_position'])
X2_min_features = combined.drop(columns=['transcript_id'])

In [52]:
X1_min_features

,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,2.355356,81.607232,885,...,False,False,False,False,False,False,False,False,False,False
1,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,3.988236,74.023667,1069,...,False,False,True,False,False,False,False,False,False,False
2,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,2.625385,88.990429,1933,...,False,False,False,True,False,False,False,False,False,False
3,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,3.803127,71.245575,678,...,False,False,True,False,False,False,False,False,False,False
4,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,3.311097,91.278961,1868,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,2.780602,84.474332,748,...,False,False,False,False,False,False,False,False,False,False
1319,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,3.420755,86.052461,711,...,False,False,False,False,False,False,False,False,False,False
1320,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,4.575894,79.646198,526,...,False,True,False,False,False,False,False,False,False,False
1321,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,2.832561,78.582196,601,...,False,False,False,False,False,False,False,False,False,False


In [50]:
model1 = pickle.load(open('xgb_model_1.pkl', 'rb'))
model2 = pickle.load(open('xgb_model_2.pkl','rb'))
X1_predicted = model1.predict_proba(X1_min_features)[:, 1]
X2_predicted = model2.predict_proba(X2_min_features)[:, 1]

In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_min_features_non_bool = X1_min_features.select_dtypes(exclude=['bool'])
X1_min_features_bool = X1_min_features.select_dtypes(include=['bool'])
x1_non_bool_scaled = scaler.fit_transform(X1_min_features_non_bool)
x1_final = np.hstack([x1_non_bool_scaled,X1_min_features_bool.values])
x1_final_proba = model1.predict_proba(x1_final)[:,1]

X2_min_features_non_bool = X2_min_features.select_dtypes(exclude=['bool'])
X2_min_features_bool = X2_min_features.select_dtypes(include=['bool'])
x2_non_bool_scaled = scaler.fit_transform(X2_min_features_non_bool)
x2_final = np.hstack([x2_non_bool_scaled,X2_min_features_bool.values])
x2_final_proba = model2.predict_proba(x2_final)[:,1]

summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = x1_final_proba.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('team1_dataset2_1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= x2_final_proba.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('team1_dataset2_2.csv',index=False)

In [97]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_scaled = scaler.fit_transform(X1_min_features)
X1_scaledpred = model1.predict_proba(X1_scaled)[:, 1]
X2_scaled = scaler.fit_transform(X2_min_features)
X2_scaledpred = model2.predict_proba(X2_scaled)[:, 1]
summary_df_scaled1 = summary_df.copy()
summary_df_scaled1['score'] = X1_scaledpred.tolist()
summary_df_scaled1[["transcript_id","transcript_position","score"]].to_csv('dataset2scaled1.csv',index=False)

summary_df_scaled2 = summary_df.copy()
summary_df_scaled2['score']= X2_scaledpred.tolist()
summary_df_scaled2[["transcript_id","transcript_position","score"]].to_csv('dataset2scaled2.csv',index=False)


In [104]:
from numpy import mean
testscore = X1_scaledpred.tolist()
mean(testscore)

0.05870082630295953

In [51]:
summary_df_final1 = summary_df.copy()
summary_df_final1['score'] =X1_predicted.tolist()

In [52]:
summary_df_final2 = summary_df.copy()
summary_df_final2['score']= X2_predicted.tolist()

In [53]:
summary_df_final1

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count,score
0,tx_id_0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,2.355356,81.607232,885,208026,0.036270
1,tx_id_0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,3.988236,74.023667,1069,208026,0.014323
2,tx_id_0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,2.625385,88.990429,1933,208026,0.022292
3,tx_id_0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,3.803127,71.245575,678,208026,0.009901
4,tx_id_0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,3.311097,91.278961,1868,208026,0.007374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,tx_id_6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,2.780602,84.474332,748,233133,0.021222
1319,tx_id_6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,3.420755,86.052461,711,233133,0.010195
1320,tx_id_6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,4.575894,79.646198,526,233133,0.023060
1321,tx_id_6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,2.832561,78.582196,601,233133,0.013053


In [54]:
summary_df_final2

,transcript_id,transcript_position,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count,score
0,tx_id_0,0,AAAACCT,0.007331,2.745006,106.466441,0.008041,4.976927,100.969944,0.006431,2.355356,81.607232,885,208026,0.106178
1,tx_id_0,10,TGGACCC,0.009909,3.410930,113.999065,0.010052,6.638470,112.420019,0.008304,3.988236,74.023667,1069,208026,0.045085
2,tx_id_0,20,GGGACTA,0.011476,3.272413,118.199172,0.008886,4.691749,118.536472,0.006855,2.625385,88.990429,1933,208026,0.226504
3,tx_id_0,30,TGGACCA,0.008315,3.507316,113.175516,0.007054,4.768628,113.163717,0.005705,3.803127,71.245575,678,208026,0.075637
4,tx_id_0,40,TAGACTA,0.008179,3.721488,119.460385,0.011026,5.393973,124.251071,0.008692,3.311097,91.278961,1868,208026,0.058376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318,tx_id_6,1840,CGAACCT,0.007091,6.525735,111.415642,0.007506,3.679412,94.022193,0.006273,2.780602,84.474332,748,233133,0.078227
1319,tx_id_6,1850,CAAACAG,0.008798,2.474871,105.786357,0.006749,3.562267,96.778762,0.009280,3.420755,86.052461,711,233133,0.093775
1320,tx_id_6,1860,CGGACAG,0.008588,3.580738,119.735741,0.007569,9.498802,113.376236,0.009389,4.575894,79.646198,526,233133,0.125545
1321,tx_id_6,1870,CAGACAA,0.007131,6.699018,106.823960,0.011142,6.307388,122.641597,0.008255,2.832561,78.582196,601,233133,0.110793


In [55]:
summary_df_final1[["transcript_id","transcript_position","score"]].to_csv('dataset2csv1.csv',index=False)

In [56]:
summary_df_final2[["transcript_id","transcript_position","score"]].to_csv('dataset2csv2.csv',index=False)